<a href="https://colab.research.google.com/github/intrinio/intrinio-tutorials/blob/master/intrinio_snowflake_query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Example using the Snowflake Python Connector
#https://dwgeek.com/how-to-use-snowflake-python-connector-and-example.html/

#Setting up environment with an environment variable 
!pip install colab-env -qU

#Installing the snowflake connector
!pip install snowflake-connector-python

#Importing those
import colab_env
import snowflake.connector

#Additional packages that are often needed for analysis
import pandas as pd
from psycopg2 import connect
import pandas.io.sql as psql
import os
from datetime import datetime
import numpy as np
from dateutil.relativedelta import relativedelta
import datetime

#The first time you run this script you need to store your credentials as environment variables
#colab_env.envvar_handler.add_env("snowflake_username", "", overwrite=True)
#colab_env.envvar_handler.add_env("snowflake_password", "", overwrite=True)
#colab_env.envvar_handler.add_env("snowflake_account", "", overwrite=True)
#colab_env.envvar_handler.add_env("snowflake_warehouse", "", overwrite=True)
#colab_env.envvar_handler.add_env("snowflake_database", "", overwrite=True)
#colab_env.envvar_handler.add_env("snowflake_schema", "", overwrite=True)

#This creates snowflake password and username, etc, from your environment variables
sf_un = os.getenv("snowflake_username")
sf_pw = os.getenv("snowflake_password")
sf_act = os.getenv("snowflake_account")
sf_wh = os.getenv("snowflake_warehouse")
sf_db = os.getenv("snowflake_database")
sf_sch = os.getenv("snowflake_schema")

In [0]:
#Connection String
conn = snowflake.connector.connect(
                user=snowflake_username,
                password=snowflake_password,
                account=snowflake_account,
                warehouse=snowflake_warehouse,
                database=snowflake_database,
                schema=snowflake_schema
                )

#Open a cursor that will move over the database? Not fully sure what this does but we need it
cur = conn.cursor()

In [0]:
#Example SQL statement
sql_statement = """SELECT * FROM STOCK_PRICE_ADJUSTMENTS_USCOMP WHERE ticker='AAPL' order by date desc LIMIT 10;"""

In [0]:
#Executing the SQL and iterating over the full result
cur.execute(sql_statement)
rows = 0
while True:
  dat = cur.fetchmany(50000)
  if not dat:
    break
  df = pd.DataFrame(dat, columns=cur.description)
  rows += df.shape[0]
print(rows)

10


In [0]:
#The line above returns multi-dimensional column names, this renames the columns with the first item in each column name

col_names = np.array([])

for sublist in cur.description:
  items = sublist[0]
  col_names = np.append(col_names, items)

In [0]:
#Adding the column names on
df.columns = col_names

In [0]:
#Closing the database connection
cur.close()
conn.close()

In [0]:
#Printing the result
df

,SECURITY_ID,COMPANY_ID,NAME,CIK,TICKER,FIGI,COMPOSITE_FIGI,COMP_TICKER,EXCH_TICKER,DATE,ADJ_FACTOR,SPLIT_RATIO,EX_DIVIDEND,EX_DIVIDEND_CURRENCY,ADJ_DIVIDEND,ADJ_DIVIDEND_RATE
0,sec_agjrgj,com_NX6GzO,Apple Inc,0000320193,AAPL,BBG000B9Y5X2,BBG000B9XRY4,AAPL:US,AAPL:UW,2020-05-08,0.997300,1.0,0.82,USD,0.82,3.90
1,sec_agjrgj,com_NX6GzO,Apple Inc,0000320193,AAPL,BBG000B9Y5X2,BBG000B9XRY4,AAPL:US,AAPL:UW,2020-02-07,0.997632,1.0,0.77,USD,0.77,3.81
2,sec_agjrgj,com_NX6GzO,Apple Inc,0000320193,AAPL,BBG000B9Y5X2,BBG000B9XRY4,AAPL:US,AAPL:UW,2019-11-07,0.997007,1.0,0.77,USD,0.77,3.77
3,sec_agjrgj,com_NX6GzO,Apple Inc,0000320193,AAPL,BBG000B9Y5X2,BBG000B9XRY4,AAPL:US,AAPL:UW,2019-08-09,0.996215,1.0,0.77,USD,0.77,3.73
4,sec_agjrgj,com_NX6GzO,Apple Inc,0000320193,AAPL,BBG000B9Y5X2,BBG000B9XRY4,AAPL:US,AAPL:UW,2019-05-10,0.996164,1.0,0.77,USD,0.77,3.69
5,sec_agjrgj,com_NX6GzO,Apple Inc,0000320193,AAPL,BBG000B9Y5X2,BBG000B9XRY4,AAPL:US,AAPL:UW,2019-02-08,0.995729,1.0,0.73,USD,0.73,3.55
6,sec_agjrgj,com_NX6GzO,Apple Inc,0000320193,AAPL,BBG000B9Y5X2,BBG000B9XRY4,AAPL:US,AAPL:UW,2018-11-08,0.996523,1.0,0.73,USD,0.73,3.45
7,sec_agjrgj,com_NX6GzO,Apple Inc,0000320193,AAPL,BBG000B9Y5X2,BBG000B9XRY4,AAPL:US,AAPL:UW,2018-08-10,0.996505,1.0,0.73,USD,0.73,2.72
8,sec_agjrgj,com_NX6GzO,Apple Inc,0000320193,AAPL,BBG000B9Y5X2,BBG000B9XRY4,AAPL:US,AAPL:UW,2018-05-11,0.996159,1.0,0.73,USD,0.73,2.62
9,sec_agjrgj,com_NX6GzO,Apple Inc,0000320193,AAPL,BBG000B9Y5X2,BBG000B9XRY4,AAPL:US,AAPL:UW,2018-02-09,0.995939,1.0,0.63,USD,0.63,2.52
